In [1]:
import neuraltda.simpComp as sc
import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import pickle
import glob
import os
reload(sa)
reload(sc)

/Users/brad/anaconda/envs/neuraltda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


<module 'neuraltda.simpComp' from '/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/simpComp.pyc'>

In [ ]:
blockPath = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/'
bdf = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/binned_data/20170128T005306Z/20170128T005306Z-10.0-5.0.binned'
scgf = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/scg/20170128T005306Z-10.0-5.0.scg'

In [ ]:
thresh = 15.0
sa.computeChainGroups(blockPath, bdf, thresh)

In [ ]:
with open(scgf, 'r') as scgff:
    scg = pickle.load(scgff)

# Bin, Compute Chain Group Generators

In [3]:
bp1 = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site01_Z3000__B1235_cat_P02_S01_1/'

bps = [bp1]

winSize = 10.0 #ms
segmentInfo = {'period': 1}
ncellsperm = 0
nperms = 0
nshuffs = 0
thresh = 6.0
propOverlap = 0.5
dtovr = propOverlap*winSize

for blockPath in bps:
    bfdict = tp2.dag_bin(blockPath, winSize, segmentInfo, ncellsperm, nperms, nshuffs, dtOverlap=dtovr)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    sa.computeChainGroups(blockPath, bdf, thresh)

Stim: A_scaled_burung Trial: 0
Stim: A_scaled_burung Trial: 1
Stim: A_scaled_burung Trial: 2
Stim: A_scaled_burung Trial: 3
Stim: A_scaled_burung Trial: 4
Stim: A_scaled_burung Trial: 5
Stim: A_scaled_burung Trial: 6
Stim: A_scaled_burung Trial: 7
Stim: A_scaled_burung Trial: 8
Stim: A_scaled_burung Trial: 9
Stim: A_scaled_burung Trial: 10
Stim: A_scaled_burung Trial: 11
Stim: A_scaled_burung Trial: 12
Stim: A_scaled_burung Trial: 13
Stim: A_scaled_burung Trial: 14
Stim: A_scaled_burung Trial: 15
Stim: A_scaled_burung Trial: 16
Stim: A_scaled_burung Trial: 17
Stim: A_scaled_burung Trial: 18
Stim: A_scaled_burung Trial: 19
Stim: B_scaled_burung Trial: 0
Stim: B_scaled_burung Trial: 1
Stim: B_scaled_burung Trial: 2
Stim: B_scaled_burung Trial: 3
Stim: B_scaled_burung Trial: 4
Stim: B_scaled_burung Trial: 5
Stim: B_scaled_burung Trial: 6
Stim: B_scaled_burung Trial: 7
Stim: B_scaled_burung Trial: 8
Stim: B_scaled_burung Trial: 9
Stim: B_scaled_burung Trial: 10
Stim: B_scaled_burung Trial:

In [ ]:
bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
sa.computeChainGroups(blockPath, bdf, thresh)

In [ ]:
len(glob.glob(os.path.join('./', '*.binned')))